### Create single table:

In [3]:
import sqlite3
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
conn = sqlite3.connect(r"D:\Coding\Data-Prep-EDA\Code\Data\online_shop.db")
transactions = pd.read_sql("SELECT * FROM TRANSACTIONS", conn)
transactions.head()

,customer,item_id,purchase_date
0,Ava,1011,4/1/23
1,Ava,1014,4/1/23
2,Ava,1015,4/15/23
3,Ava,1018,5/1/23
4,Ben,2345,4/15/23


In [6]:
items = pd.read_sql("SELECT * FROM ITEMS", conn)
items.head()

,item_id,item_description,price,category,rating
0,1011,Paint,$15.99,Arts & Crafts,3.5
1,1012,Crayons,$2.87,Arts & Crafts,4.7
2,1013,Markers,$2.50,Arts & Crafts,4.8
3,1014,Brush,$1.99,Arts & Crafts,4.2
4,1015,Paper,$22.49,Arts & Crafts,4.5


In [7]:
df = transactions.merge(items, how="left", on="item_id")
df.head()

,customer,item_id,purchase_date,item_description,price,category,rating
0,Ava,1011,4/1/23,Paint,$15.99,Arts & Crafts,3.5
1,Ava,1014,4/1/23,Brush,$1.99,Arts & Crafts,4.2
2,Ava,1015,4/15/23,Paper,$22.49,Arts & Crafts,4.5
3,Ava,1018,5/1/23,Scissors,$3.50,Arts & Crafts,4.6
4,Ben,2345,4/15/23,Dog Food,$29.99,Pet Supplies,4.9


In [8]:
df.shape

(110, 7)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customer          110 non-null    object
 1   item_id           110 non-null    int64 
 2   purchase_date     110 non-null    object
 3   item_description  110 non-null    object
 4   price             110 non-null    object
 5   category          110 non-null    object
 6   rating            110 non-null    object
dtypes: int64(1), object(6)
memory usage: 6.1+ KB


### Prepare rows for modelling: